In [1]:
import boto3
import pandas as pd
import numpy as np
from io import StringIO


In [2]:
# Data posted by Michael Roy on Data World
# https://data.world/michaelaroy/ncaa-tournament-results
raw_df = pd.read_csv('https://query.data.world/s/gp7t4y3y3bcmazezh3hyfa64hjed6j')

### Transform tournament data into a long format 

#### Old format

In [3]:
raw_df.head()

,Year,Round,Region Number,Region Name,Seed,Score,Team,Team.1,Score.1,Seed.1
0,1985,1,1,West,1,83,St Johns,Southern,59,16
1,1985,1,1,West,2,81,VCU,Marshall,65,15
2,1985,1,1,West,3,65,NC State,Nevada,56,14
3,1985,1,1,West,4,85,UNLV,San Diego St,80,13
4,1985,1,1,West,5,58,Washington,Kentucky,65,12


In [4]:
base_columns = ['Year','Round','Region Number','Region Name']
team0_columns = ['Team','Seed','Score']
team1_columns = ['Team.1','Seed.1','Score.1']

In [5]:
high_seed_df = raw_df[base_columns + team0_columns].copy()
low_seed_df  = raw_df[base_columns + team1_columns].rename(columns=dict(zip(team1_columns,team0_columns))).copy()

high_seed_df['Team_Position'] = 'high'
low_seed_df['Team_Position'] = 'low'
raw_long_df = pd.concat([high_seed_df, low_seed_df])

#### New format

In [7]:
raw_long_df.head()

,Year,Round,Region Number,Region Name,Team,Seed,Score,Team_Position
0,1985,1,1,West,St Johns,1,83,high
1,1985,1,1,West,VCU,2,81,high
2,1985,1,1,West,NC State,3,65,high
3,1985,1,1,West,UNLV,4,85,high
4,1985,1,1,West,Washington,5,58,high


### Fix data issue with 2019 regions that are flipped

In this data, the 2019 rounds 2 - final four East and South regions are flipped. I checked the rest of the data and this is the only error of this kind. You can see in round 1 Duke is in the East and Virginia is in the South. And in rounds 2-4 that is flipped.

In [11]:
is2019 = raw_long_df.Year == 2019
seed1 = raw_long_df.Seed == 1
bad_regions = raw_long_df['Region Name'].isin(['East','South'])
raw_long_df[is2019 & seed1 & bad_regions]

,Year,Round,Region Number,Region Name,Team,Seed,Score,Team_Position
2142,2019,1,1,East,Duke,1,85,high
2158,2019,1,3,South,Virginia,1,71,high
2174,2019,2,1,South,Duke,1,77,high
2182,2019,2,3,East,Virginia,1,63,high
2190,2019,3,1,South,Duke,1,75,high
2194,2019,3,3,East,Virginia,1,53,high
2198,2019,4,1,South,Duke,1,67,high
2200,2019,4,3,East,Virginia,1,80,high


Here we look at how bad that problem is. But it is limited to 2019 East and South regions.      
This table shows the number of regions that each team is in during each year.

In [12]:
regional_rounds = raw_long_df.Round <= 4 # Filter out where region name is Final Four or Championship
raw_long_df[regional_rounds] \
    .groupby(['Year','Region Name','Team']) \
    .count() \
    .reset_index() \
    .groupby(['Year','Team'])['Region Name'].count() \
    .reset_index() \
    .sort_values('Region Name', ascending=False) \
    .head(20)

,Year,Team,Region Name
2200,2019,LSU,2
2221,2019,Oregon,2
2204,2019,Maryland,2
2201,2019,Liberty,2
2218,2019,Oklahoma,2
2227,2019,Tennessee,2
2183,2019,Central Florida,2
2229,2019,UC Irvine,2
2206,2019,Michigan St,2
2207,2019,Minnesota,2


Get a boolean mask for the rows that need to be changed

In [13]:
is2019 = (raw_long_df.Year == 2019)
isBadRound = (raw_long_df.Round > 1)
isEast = (raw_long_df['Region Name'] == 'East')
isSouth = (raw_long_df['Region Name'] == 'South')

badEastRows = (is2019 & isBadRound & isEast)
badSouthRows = (is2019 & isBadRound & isSouth)

Correct the bad data

In [14]:
raw_long_df.loc[badEastRows,'Region Name'] = 'South'
raw_long_df.loc[badSouthRows,'Region Name'] = 'East'

### Create some standardization on region names

The region names change year to year. I wanted to have some standardization in region names...     
I hope this doesn't make things confusing later.     
Below shows the relative number of years each region shows up.

In [15]:
raw_long_df.groupby('Region Name').count()

,Year,Round,Region Number,Team,Seed,Score,Team_Position
Region Name,,,,,,,
Championship,70,70,70,70,70,70,70
East,1050,1050,1050,1050,1050,1050,1050
Final Four,140,140,140,140,140,140,140
Midwest,1020,1020,1020,1020,1020,1020,1020
South,630,630,630,630,630,630,630
Southeast,420,420,420,420,420,420,420
Southwest,30,30,30,30,30,30,30
West,1050,1050,1050,1050,1050,1050,1050


East and West are there every year.     
Change Southwest --> Midwest     
and Southeast --> South      
and then Region Names will be consistent across years
I guess not that it matters since there is also a region number     

In [16]:
isSoutheast = raw_long_df['Region Name'] == 'Southeast'
isSouthwest = raw_long_df['Region Name'] == 'Southwest'

In [17]:
raw_long_df.loc[isSoutheast,'Region Name'] = 'South'
raw_long_df.loc[isSouthwest,'Region Name'] = 'Midwest'

In [18]:
raw_long_df.groupby('Region Name').count()

,Year,Round,Region Number,Team,Seed,Score,Team_Position
Region Name,,,,,,,
Championship,70,70,70,70,70,70,70
East,1050,1050,1050,1050,1050,1050,1050
Final Four,140,140,140,140,140,140,140
Midwest,1050,1050,1050,1050,1050,1050,1050
South,1050,1050,1050,1050,1050,1050,1050
West,1050,1050,1050,1050,1050,1050,1050


In [19]:
tournament_df = raw_long_df.reset_index().copy()

#### Figure out which regions play each other in the final four because it changes from year to year

In [20]:
final_four_games = \
tournament_df[tournament_df['Round'] == 4] \
    .assign(game_num = lambda x: (x['Region Number'] <= 2)+1) \
    .groupby(['Year','game_num']) \
    .agg({'Region Name' : lambda x: dict(zip(['team1','team2'],x))}) \
    .groupby('Year') \
    .agg({'Region Name' : lambda x: dict(zip(['game1','game2'],x))}) \
    .to_dict().get('Region Name')
#{1985: {'game1': {'team1': 'Midwest', 'team2': 'South'}, 'game2': {'team1': 'West', 'team2': 'East'}},
# 1986: {'game1': {'team1': 'East', 'team2': 'Midwest'}, 'game2': {'team1': 'South', 'team2': 'West'}},
# 1987: {'game1': {'team1': 'South', 'team2': 'East'}, 'game2': {'team1': 'West', 'team2': 'Midwest'}},
#        ...

#### Create dictionary file with all data in easy format for Bracket object

In [53]:
years = tournament_df.Year.unique().tolist()

brackets_dict = {}
for year in years:
    that_year_df = tournament_df[tournament_df['Year'] == year].copy()
    round1 = that_year_df[that_year_df['Round'] == 1].sort_values('Seed')
    round2 = that_year_df[that_year_df['Round'] == 2]
    round3 = that_year_df[that_year_df['Round'] == 3]
    round4 = that_year_df[that_year_df['Round'] == 4]
    round5 = that_year_df[that_year_df['Round'] == 5]
    round6 = that_year_df[that_year_df['Round'] == 6]
    winner = round6.sort_values('Score', ascending=False).reset_index().loc[0,].to_dict()
    
    east1 = round1[round1['Region Name'] == 'East']
    west1 = round1[round1['Region Name'] == 'West']
    midwest1 = round1[round1['Region Name'] == 'Midwest']
    south1 = round1[round1['Region Name'] == 'South']

    results = {
        'first' : round1[['Team','Seed']].to_dict('recods'),
        'second' : round2[['Team','Seed']].to_dict('recods'),
        'sweet16' : round3[['Team','Seed']].to_dict('recods'),
        'elite8' : round4[['Team','Seed']].to_dict('recods'),
        'final4' : round5[['Team','Seed']].to_dict('recods'),
        'championship' : round6[['Team','Seed']].to_dict('recods'),
        'winner' : {'Team' : winner['Team'], 'Seed' : int(winner['Seed'])}
    }
    regions = {
        'East' : east1[['Team','Seed']].to_dict('recods'),
        'West' : west1[['Team','Seed']].to_dict('recods'),
        'Midwest' : midwest1[['Team','Seed']].to_dict('recods'),
        'South' : south1[['Team','Seed']].to_dict('recods')
    }
    brackets_dict[year] = {'Region' : regions, 'Results': results, 'Finals': final_four_games[year]}


In [54]:
import json
with open('brackets.json' , 'w') as f:
    json.dump(brackets_dict, f)

In [55]:
import json
with open('brackets.json', 'r') as f:
    brackets_dict = json.load(f)


In [57]:
brackets_dict['2019']['Results']['winner']

{'Team': 'Virginia', 'Seed': 1}

In [ ]:
# Write the cleaned data to S3
import boto3
from io import StringIO
bucket_name = 'ncaabb-data'
file_name = 'tournament.csv'

boto3.resource('s3').Bucket(bucket_name).Object(file_name).put(Body=tournament_df.to_csv(index=False))

In [ ]:
# Check to make sure that the data is there

byte_str = boto3.resource('s3').Bucket(bucket_name).Object(file_name).get()['Body'].read().decode('utf-8')
clean_df = pd.read_csv(StringIO(str(byte_str)))
(tournament_df==clean_df).mean().mean()

In [ ]:
bucket_name = 'ncaabb-data'
tournament_filename = 'big_dance.csv'
season19_filename = 'team19.csv'

In [ ]:
s3 = boto3.resource('s3')
s3_bucket = s3.Bucket(bucket_name)

In [ ]:
byte_str = s3_bucket.Object(tournament_filename).get()['Body'].read().decode('utf-8')
tournament_raw_df = pd.read_csv(StringIO(str(byte_str)))

In [ ]:
byte_str = s3_bucket.Object(season19_filename).get()['Body'].read().decode('utf-8')
season19_df = pd.read_csv(StringIO(str(byte_str)))

In [ ]:
base_columns = ['Year','Round','Region Number','Region Name']
team0_columns = ['Team','Seed','Score']
team1_columns = ['Team.1','Seed.1','Score.1']

In [ ]:
high_seed_df = tournament_raw_df[base_columns + team0_columns].copy()
low_seed_df  = tournament_raw_df[base_columns + team1_columns].rename(columns=dict(zip(team1_columns,team0_columns))).copy()

high_seed_df['Team_Position'] = 'high'
low_seed_df['Team_Position'] = 'low'
tournament_df = pd.concat([high_seed_df, low_seed_df])

In [ ]:
final_four_games

In [ ]:
teams = pd.concat([tourney19['Team'], tourney19['Team.1']]).sort_values().unique().tolist()

In [ ]:
# Filter on schools that made it to the tourney
season19_df = season19_df[season19_df.School.str.contains('NCAA')].copy()

# Take out the NCAA tag on the team name
season19_df['School'] = season19_df.School.str.replace(' NCAA', '')

# Manually rename teams with more colloquial acronyms
season19_df.loc[season19_df['School'] == 'Virginia Commonwealth', 'School'] = 'VCU'
season19_df.loc[season19_df['School'] == 'Louisiana State', 'School'] = 'LSU'
season19_df.loc[season19_df['School'] == 'Mississippi', 'School'] = 'Ole Miss'

# Take out play in game losers
play_in_losers = ['Temple','North Carolina Central',"St. John's (NY)",'Prairie View']
lost_before_64 = season19_df.School.isin(play_in_losers)
season19_df = season19_df[~lost_before_64].copy()

In [ ]:
#! conda install -c conda-forge fuzzywuzzy -y

In [ ]:
season19_df.head(64)

In [ ]:
teams_from_season_data = season19_df.School.values.tolist()

In [ ]:
len(teams_from_season_data)

In [ ]:
def get_match(string, potential_matches):
    best_match, score = process.extractOne(string, potential_matches, scorer=fuzz.token_sort_ratio)
    if score < 80: print(string, best_match, score)#raise Exception("No good match with score above 80")
    return best_match

In [ ]:
team_name_map = {school:get_match(school, teams) for school in teams_from_season_data}
season19_df['School'] = season19_df['School'].map(team_name_map)

In [ ]:
expected_wins_for_seed = {
#  seed : expected wins    
    16 : 0,
    15 : 0,
    14 : 0,
    13 : 0,
    12 : 0,
    11 : 0,
    10 : 0,
    9  : 0,
    8  : 1,
    7  : 1,
    6  : 1,
    5  : 1,
    4  : 2,
    3  : 2,
    2  : 3,
    1  : 4
}

In [ ]:
# how to structure what happened in tournament
# how did each seed in each region (or each team) perform ?
the_team = 'Duke'
tourney19[(tourney19['Team'] == the_team) | (tourney19['Team.1'] == the_team)]

In [ ]:
tourney19[tourney19['Region Name'] == 'East']

In [ ]:
tournament_df.groupby('Region Name').count()